In [3]:
from keras.layers import *
from keras.models import Model
from keras import layers
from keras.layers.merge import concatenate
import sys
sys.path.insert(1, '../src')
sys.path.insert(1, '../image_segmentation_keras')
from keras_segmentation.models.config import IMAGE_ORDERING

from keras_segmentation.models.model_utils import get_segmentation_model
from glob import glob
from crfrnn_layer import CrfRnnLayer

Using TensorFlow backend.


In [4]:
input_height = 256
input_width = 256
n_classes = 3
channels = 3

In [18]:
def unet_conv_block(inputs, filters, pool=True, batch_norm_first=True):
    if batch_norm_first == True:
        input_ = Conv2D(filters, 3, padding="same")(inputs)
        print(input_.shape)
        x = BatchNormalization()(input_)
        x = Activation("relu")(x)

        x = Conv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
        x = CrfRnnLayer(image_dims=(input_height, input_width),
                         num_classes=n_classes,
                         theta_alpha=160.,
                         theta_beta=3.,
                         theta_gamma=3.,
                         num_iterations=10,
                         name='crfrnn')([x, input_])
    elif batch_norm_first == False:
        input_ = Conv2D(filters, 3, padding="same")(inputs)
        print(input_.shape)
        x = Activation("relu")(input_)
        x = BatchNormalization()(x)

        x = Conv2D(filters, 3, padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        print(x.shape)
        x = CrfRnnLayer(image_dims=(input_height, input_width),
                         num_classes=filters,
                         theta_alpha=160.,
                         theta_beta=3.,
                         theta_gamma=3.,
                         num_iterations=10,
                         name='crfrnn')([x, input_])
        print(x.shape)
    if pool == True:
        p = MaxPooling2D((2, 2))(x)
        return [x, p]
    else:
        return x

In [19]:
img_input = Input(shape=(input_height,input_width, channels))
f1 = unet_conv_block(img_input, 64, pool=True, batch_norm_first=False)
f2 = unet_conv_block(f1[1], 128, pool=True, batch_norm_first=False)
f3 = unet_conv_block(f2[1], 256, pool=True, batch_norm_first=False)
f4 = unet_conv_block(f3[1], 512, pool=True, batch_norm_first=False)
f5 = unet_conv_block(f4[1], 1024, pool=False, batch_norm_first=False)

x = UpSampling2D((2, 2))(f5)
x = concatenate([x, f4[0]], axis=3)
x = unet_conv_block(x, 512, pool=False, batch_norm_first=False)

x = UpSampling2D((2, 2))(x)
x = concatenate([x, f3[0]], axis=3)
x = unet_conv_block(x, 256, pool=False, batch_norm_first=False)

x = UpSampling2D((2, 2))(x)
x = concatenate([x, f2[0]], axis=3)
x = unet_conv_block(x, 128, pool=False, batch_norm_first=False)

x = UpSampling2D((2, 2))(x)
x = concatenate([x, f1[0]], axis=3)
x = unet_conv_block(x, 64, pool=False, batch_norm_first=False)

x = Conv2D(n_classes, (3, 3), padding='same')(x)

# crf_output = CrfRnnLayer(image_dims=(input_height, input_width),
#                          num_classes=n_classes,
#                          theta_alpha=160.,
#                          theta_beta=3.,
#                          theta_gamma=3.,
#                          num_iterations=10,
#                          name='crfrnn')([x, img_input])
model = get_segmentation_model(img_input, crf_output)


(None, 256, 256, 64)
(None, 256, 256, 64)
(1, 256, 256, 64)
(1, 128, 128, 128)
(1, 128, 128, 128)


ValueError: Dimensions must be equal, but are 128 and 256 for '{{node crfrnn_12/truediv}} = RealDiv[T=DT_FLOAT](crfrnn_12/HighDimFilter_2, crfrnn_12/HighDimFilter)' with input shapes: [128,128,128], [128,256,256].

In [5]:
model.load_weights('/Users/mavaylon/Research/pet_weights/UNET_CRF_PET/pet_class_crf.h5')

In [6]:
model.train(
    train_images =  "/Users/mavaylon/Research/Data1/train/img/",
    train_annotations = "/Users/mavaylon/Research/Data1/train/ann/",
    epochs=20,
    steps_per_epoch=len(glob("/Users/mavaylon/Research/Data1/train/img/*")),
    batch_size=1,
    validate=True,
    val_images="/Users/mavaylon/Research/Data1/test/img/",
    val_annotations="/Users/mavaylon/Research/Data1/test/ann/",
    val_batch_size=1,
    val_steps_per_epoch=len(glob("/Users/mavaylon/Research/Data1/test/img/*"))
)

Verifying training dataset


  2%|▏         | 34/1478 [00:00<00:04, 330.72it/s]

Dataset verified! 
Verifying validation dataset


100%|██████████| 1478/1478 [00:04<00:00, 334.25it/s]


Dataset verified! 
fit
Epoch 1/20
5912/5912 [==============================] - 23981s 4s/step - loss: 0.6582 - accuracy: 0.7401 - val_loss: 0.4767 - val_accuracy: 0.7938

Epoch 00001: val_accuracy improved from -inf to 0.79382, saving model to pet_class_crf.h5
Epoch 2/20
5912/5912 [==============================] - 23862s 4s/step - loss: 0.5146 - accuracy: 0.8069 - val_loss: 0.4435 - val_accuracy: 0.8066

Epoch 00002: val_accuracy improved from 0.79382 to 0.80659, saving model to pet_class_crf.h5
Epoch 3/20
5912/5912 [==============================] - 23481s 4s/step - loss: 0.4508 - accuracy: 0.8338 - val_loss: 0.4289 - val_accuracy: 0.7916

Epoch 00003: val_accuracy did not improve from 0.80659
Epoch 4/20
5912/5912 [==============================] - 23406s 4s/step - loss: 0.4062 - accuracy: 0.8519 - val_loss: 0.5158 - val_accuracy: 0.8135

Epoch 00004: val_accuracy improved from 0.80659 to 0.81347, saving model to pet_class_crf.h5
Epoch 5/20
5912/5912 [==============================] 